In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
import numpy as np

train_df = pd.read_pickle("../data/train_cens.pkl.gz")
train_df.head()

,name,parent,cens_slice,index
0,XC109605,acafly,"[[0.3086012466109237, 0.30568218743577036, 0.3...","(25, 75)"
0,XC109605,acafly,"[[0.37812375487169103, 0.3725999263434882, 0.3...","(157, 207)"
0,XC109605,acafly,"[[0.38193222855739883, 0.3810666924182529, 0.3...","(260, 310)"
0,XC109605,acafly,"[[0.3523993330368658, 0.3480284909351147, 0.34...","(324, 374)"
0,XC109605,acafly,"[[0.3179275282653602, 0.3194400852268083, 0.32...","(361, 411)"


In [2]:
x = train_df.cens_slice.iloc[0]

In [3]:
pad = np.zeros(12*64).reshape(12, -1)
pad[:, :x.shape[1]] = x
pad.reshape(12, 8, 8).T.shape

(8, 8, 12)

In [4]:
import tensorflow as tf

def reshape_square(arr, k=8):
    """Domain specific. Transform 12x50 matrix into a square matrix that's 25x25."""
    x = np.zeros((12, k*k))
    x[:, :arr.shape[1]] = arr
    return x.reshape(12, k, k).T
    
X = np.stack(train_df.cens_slice.apply(reshape_square))

lb = LabelBinarizer()
lb.fit(train_df.parent)
y = lb.transform(train_df.parent)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [5]:
# https://github.com/tensorpack/tensorpack/blob/5845ead80e3d77c89e2f1e9d5e0691324a8354cc/examples/ResNet/cifar10-preact18-mixup.py#L101-L118

def _mixup(x_in, y_in):
    n = x_in.shape[0]
    # draw from uniform instead of beta(alpha, alpha, BATCH_SIZE)
    w = np.random.uniform(0.3, 0.7, size=n)
    x_weight = w.reshape(n, 1, 1, 1)
    index = np.random.permutation(n)

    x = x_in * x_weight + x_in[index] * (1-x_weight)
    y = y_in + y_in[index]
    return x, y

@tf.function(input_signature=[
    tf.TensorSpec(None, tf.float64), 
    tf.TensorSpec(None, tf.int32), 
])
def mixup(x_in, y_in):
    x, y = tf.numpy_function(_mixup, [x_in, y_in], [tf.float64, tf.int32])
    return tf.data.Dataset.from_tensor_slices((
        tf.reshape(x, [-1, 8, 8, 12]), 
        tf.reshape(y, [-1, 397])
    ))

def add_mixup(dataset, batch_size=1024):
    return (
        dataset
        .shuffle(batch_size)
        .window(batch_size)
        .flat_map(lambda x, y: tf.data.Dataset.zip((x.batch(batch_size), y.batch(batch_size))))
        .map(mixup, num_parallel_calls=tf.data.AUTOTUNE)
        .flat_map(lambda x: x)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

add_mixup(test_dataset)

<PrefetchDataset shapes: ((None, 8, 8, 12), (None, 397)), types: (tf.float64, tf.int32)>

In [6]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Softmax, Flatten, Dense
from tensorflow_addons.metrics import F1Score

tf.config.list_physical_devices('GPU')
def get_compiled_model(input_shape=(8, 8, 12), output_shape=397):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=input_shape))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))

    model.add(Flatten())
    model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(output_shape, activation='sigmoid'))
    
    model.compile(optimizer='adam',
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        metrics=['accuracy', F1Score(num_classes=output_shape)])
    return model

In [7]:
model = get_compiled_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 8, 8, 32)          3488      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 8, 8, 32)          9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 4, 4, 32)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 4, 4, 64)          18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 64)          0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 128)         7

In [ ]:
history = model.fit(add_mixup(train_dataset, 128), epochs=100, validation_data=test_dataset.batch(128))

Epoch 1/100
3450/3450 [==============================] - 64s 16ms/step - loss: 0.0404 - accuracy: 0.0165 - f1_score: 0.0014 - val_loss: 0.0174 - val_accuracy: 0.0214 - val_f1_score: 0.0013e
Epoch 2/100
3450/3450 [==============================] - 63s 18ms/step - loss: 0.0302 - accuracy: 0.0251 - f1_score: 0.0024 - val_loss: 0.0174 - val_accuracy: 0.0259 - val_f1_score: 0.0025
Epoch 3/100
3450/3450 [==============================] - 85s 25ms/step - loss: 0.0300 - accuracy: 0.0262 - f1_score: 0.0032 - val_loss: 0.0170 - val_accuracy: 0.0286 - val_f1_score: 0.0039
Epoch 4/100
3450/3450 [==============================] - 115s 33ms/step - loss: 0.0299 - accuracy: 0.0286 - f1_score: 0.0045 - val_loss: 0.0170 - val_accuracy: 0.0325 - val_f1_score: 0.0056
Epoch 5/100
3450/3450 [==============================] - 153s 44ms/step - loss: 0.0298 - accuracy: 0.0292 - f1_score: 0.0059 - val_loss: 0.0169 - val_accuracy: 0.0343 - val_f1_score: 0.0063
Epoch 6/100
3450/3450 [=============================

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

plt.plot([x.mean() for x in history.history['f1_score']], label='train')
plt.plot([x.mean() for x in history.history['val_f1_score']], label='test')
plt.xlabel('Epoch')
plt.ylabel('F1 Score')
plt.legend()
plt.show()

plt.plot(history.history["loss"], label="train")
plt.plot(history.history["val_loss"], label="test")
plt.xlabel("Epoch")
plt.ylabel("loss")
plt.legend()
plt.show()

model.evaluate(X_test, y_test, verbose=2)

## appendix

In [8]:
history = model.fit(add_mixup(train_dataset, 256), epochs=5, validation_data=test_dataset.batch(256))

Epoch 1/5
1725/1725 [==============================] - 33s 16ms/step - loss: 0.0492 - accuracy: 0.0163 - f1_score: 7.8527e-04 - val_loss: 0.0175 - val_accuracy: 0.0129 - val_f1_score: 3.3404e-04
Epoch 2/5
1725/1725 [==============================] - 25s 15ms/step - loss: 0.0303 - accuracy: 0.0224 - f1_score: 0.0017 - val_loss: 0.0174 - val_accuracy: 0.0237 - val_f1_score: 0.0023
Epoch 3/5
1725/1725 [==============================] - 25s 14ms/step - loss: 0.0301 - accuracy: 0.0269 - f1_score: 0.0028 - val_loss: 0.0172 - val_accuracy: 0.0279 - val_f1_score: 0.0030
Epoch 4/5
1725/1725 [==============================] - 26s 15ms/step - loss: 0.0299 - accuracy: 0.0274 - f1_score: 0.0037 - val_loss: 0.0171 - val_accuracy: 0.0287 - val_f1_score: 0.0040
Epoch 5/5
1725/1725 [==============================] - 26s 15ms/step - loss: 0.0298 - accuracy: 0.0289 - f1_score: 0.0047 - val_loss: 0.0171 - val_accuracy: 0.0324 - val_f1_score: 0.0058


In [10]:
history = model.fit(add_mixup(train_dataset, 128), epochs=5, validation_data=test_dataset.batch(128))

Epoch 1/5
3450/3450 [==============================] - 45s 13ms/step - loss: 0.0298 - accuracy: 0.0291 - f1_score: 0.0061 - val_loss: 0.0170 - val_accuracy: 0.0350 - val_f1_score: 0.0075
Epoch 2/5
3450/3450 [==============================] - 45s 13ms/step - loss: 0.0297 - accuracy: 0.0313 - f1_score: 0.0073 - val_loss: 0.0168 - val_accuracy: 0.0378 - val_f1_score: 0.0089
Epoch 3/5
3450/3450 [==============================] - 45s 13ms/step - loss: 0.0297 - accuracy: 0.0328 - f1_score: 0.0084 - val_loss: 0.0169 - val_accuracy: 0.0386 - val_f1_score: 0.0096
Epoch 4/5
3450/3450 [==============================] - 45s 13ms/step - loss: 0.0296 - accuracy: 0.0337 - f1_score: 0.0093 - val_loss: 0.0168 - val_accuracy: 0.0410 - val_f1_score: 0.0109
Epoch 5/5
3450/3450 [==============================] - 45s 13ms/step - loss: 0.0296 - accuracy: 0.0342 - f1_score: 0.0099 - val_loss: 0.0168 - val_accuracy: 0.0411 - val_f1_score: 0.0110


In [11]:
history = model.fit(add_mixup(train_dataset, 64), epochs=2, validation_data=test_dataset.batch(64))

Epoch 1/2
6900/6900 [==============================] - 115s 17ms/step - loss: 0.0296 - accuracy: 0.0337 - f1_score: 0.0097 - val_loss: 0.0168 - val_accuracy: 0.0415 - val_f1_score: 0.0117
Epoch 2/2
6900/6900 [==============================] - 187s 27ms/step - loss: 0.0296 - accuracy: 0.0341 - f1_score: 0.0101 - val_loss: 0.0168 - val_accuracy: 0.0419 - val_f1_score: 0.0121
